In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

In [4]:
df = pd.read_parquet("escooter_history_2022_new.parquet.gzip")

In [5]:
df_agg = df.groupby(["date","hour", "weekday", "daytype"], as_index=False).agg(
    total_rental=("datetime","count"),
    temp_mode=("temp", pd.Series.mode),
    temp_mean=("temp", "mean"),
    atemp_mode=("atemp", pd.Series.mode),
    atemp_mean=("atemp", "mean"),
    humidity_mode=("humidity", pd.Series.mode),
    humidity_mean=("humidity", "mean"),
    weather_mode=("weather", pd.Series.mode),
    windspeed_mode=("windspeed", pd.Series.mode),
    windspeed_mean=("windspeed", "mean"),
    total_registered=("registered_customer", "sum")
    ).round()
df_agg["total_unregistered"] = df_agg["total_rental"] - df_agg["total_registered"] 
df_agg['day_hour'] = (pd.to_datetime(df_agg['date'], dayfirst=False)
               + pd.to_timedelta(df_agg['hour'].astype(int), unit='H')
              )

In [6]:
def normalized_df(param, aggregation, df=df_agg):
    """
    Funktion, um Dataframes zu erstellen, die gruppiert nach 
    einer Attribut des df_agg bspw. temp_mode (param_aggregation) die durchschnittliche Anzahl
    Ausleihen pro Stunde je Attribut-Ausprägung enthält
    Beispielsweise:
    Bei Temperatur 30 Grad gibt es 200 Ausleihen pro Stunde durchschnittlich 
    
    """
    df_new = df.groupby(f"{param}_{aggregation}", as_index=False).agg(
        count = ("date", "count"),
        total_rental = ("total_rental", "sum"),
        total_registered = ("total_registered", "sum"),
        total_unregistered = ("total_unregistered", "sum"),
        mean_rental_hour = ("total_rental", "mean"),
        mean_rental_hour_r = ("total_registered", "mean"),
        mean_rental_hour_u = ("total_unregistered", "mean")

    )
    # df_new[f"mean_rental_per_{param}_hour"] = df_new["total_rental"] / df_new["count"]
    # df_new[f"mean_rental_per_{param}_hour_r"] = df_new["total_registered"] / df_new["count"]
    # df_new[f"mean_rental_per_{param}_hour_u"] = df_new["total_unregistered"] / df_new["count"]  
 
    return df_new


def pearson_for_df(df,param, aggregation):
    """
    Errechnet und printed für ein df der Form von normalized_df den
    Pearson-Koeffizient für total, registered und unregistered.
    Übergeben werden muss param (bpsw. temp) und aggregation (bspw. mode)
    des Dataframes    
    """
    total = pearsonr(df[f"{param}_{aggregation}"], df[f"mean_rental_per_{param}_hour"])
    registered = pearsonr(df[f"{param}_{aggregation}"], df[f"mean_rental_per_{param}_hour_r"])
    unregistered = pearsonr(df[f"{param}_{aggregation}"], df[f"mean_rental_per_{param}_hour_u"])
    print(f"total: {total}")
    print(f"registered: {registered}")
    print(f"unregistered: {unregistered}")

In [7]:
# Für alle Attribut wird ein DF erstellt
df_temp_mean = normalized_df("temp", "mean")
df_temp_mode = normalized_df("temp", "mode")
df_atemp_mean = normalized_df("atemp", "mean")
df_atemp_mode = normalized_df("atemp", "mode")
df_humidity_mean = normalized_df("humidity", "mean")
df_humidity_mode = normalized_df("humidity", "mode")
df_windspeed_mean = normalized_df("windspeed", "mean")
df_windspeed_mode = normalized_df("windspeed", "mode")
df_weather_mode = normalized_df("weather", "mode")

In [8]:
px.scatter(df_temp_mode, x= "temp_mode", y="mean_rental_per_temp_hour", color="count", trendline="ols", trendline_color_override="darkgrey")

In [9]:
pearson_for_df(df_temp_mode, "temp", "mode")

total: (0.9186450023193855, 2.5669977225895685e-17)
registered: (0.8778808046286714, 4.8517910854365866e-14)
unregistered: (0.9535348298022555, 6.356756278772309e-22)


In [10]:
px.scatter(df_temp_mean, x= "temp_mean", y="mean_rental_per_temp_hour", color="count", trendline="ols", trendline_color_override="darkgrey")

In [11]:
pearson_for_df(df_temp_mean, "temp", "mean")

total: (0.9187359588021505, 2.5136966552281732e-17)
registered: (0.8780719815524128, 4.7141920393582373e-14)
unregistered: (0.9533750554754542, 6.787070037176629e-22)


In [12]:
px.scatter(df_atemp_mode, x= "atemp_mode", y="mean_rental_per_atemp_hour", color="count", hover_name="count", trendline="ols", trendline_color_override="darkgrey")

In [13]:
pearson_for_df(df_atemp_mode, "atemp", "mode")

total: (0.7775145444450163, 3.115942788996269e-11)
registered: (0.7504697836185983, 3.4985457034763896e-10)
unregistered: (0.8139078472882224, 6.674182255885447e-13)


In [14]:
px.scatter(df_atemp_mean, x= "atemp_mean", y="mean_rental_per_atemp_hour", color="count", hover_name="count", trendline="ols", trendline_color_override="darkgrey")

In [15]:
pearson_for_df(df_atemp_mean, "atemp", "mean")

total: (0.777625251580012, 3.083137646947493e-11)
registered: (0.750813074343052, 3.399355666124126e-10)
unregistered: (0.814114957772372, 6.514441529791276e-13)


In [16]:
px.scatter(df_windspeed_mode, x= "windspeed_mode", y="mean_rental_per_windspeed_hour", color="count", trendline="ols", trendline_color_override="darkgrey")

In [17]:
pearson_for_df(df_windspeed_mode, "windspeed", "mode")

total: (-0.4249401387870084, 0.019243882046049203)
registered: (-0.6000807663608998, 0.00045603480525607234)
unregistered: (0.054430717451683125, 0.7751285270274856)


In [18]:
px.scatter(df_windspeed_mean, x= "windspeed_mean", y="mean_rental_per_windspeed_hour", color="count", trendline="ols", trendline_color_override="darkgrey")

In [19]:
pearson_for_df(df_windspeed_mean, "windspeed", "mean")

total: (0.043451371235926406, 0.766888230368732)
registered: (-0.05094579847494497, 0.728110400364267)
unregistered: (0.25980683358156387, 0.07142015612239704)


In [20]:
px.bar(df_weather_mode, x="weather_mode", y="mean_rental_per_weather_hour", color="count")